In [1]:
import polars as pl
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import pdb
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from featurewiz_polars import print_classification_metrics, print_regression_metrics
n_features = 20
n_informative = 10
#X, y = make_classification(n_samples = 5000, n_features = n_features, 
#                n_informative = n_informative, n_redundant=4, random_state=42)
datapath = "../../../ram/Documents/Ram/data_sets/"
filename = "horse.csv"
df = pl.read_csv(datapath+filename, null_values='NULL', try_parse_dates=True)#.sample(1000)
print('Loaded data...', df.shape)
target = 'target' # Replace with your target column name
model_type = 'Classification'
if target not in df.columns:
    print(f"Error: Target column '{target}' not found in the CSV file.")
    exit()
predictors = [x for x in df.columns if x!=target]
X = df[predictors]
y = df[target]
print('Data dimensions (rows x cols) = %d dims' %(int(X.shape[0]*X.shape[1])))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train.dtype

Loaded data... (299, 28)
Data dimensions (rows x cols) = 8073 dims


String

In [2]:
import time
from featurewiz_polars import Featurewiz_MRMR_Model, Featurewiz_MRMR
from sklearn.pipeline import Pipeline
from polars_other_transformers import Polars_ColumnEncoder, YTransformer
from polars_other_transformers import Polars_MissingTransformer
from polars_categorical_encoder import Polars_CategoricalEncoder # Now using V2 of Encoder
from polars_datetime_transformer import Polars_DateTimeTransformer # Import new transformer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier # Or LightGBM, XGBoost, etc.
from polars_sulov_mrmr import Sulov_MRMR
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier # Or LightGBM, XGBoost, etc.
model = RandomForestClassifier(n_estimators=100, random_state=99)
#testp = Featurewiz_MRMR(model_type=model_type, encoding_type='ordinal')
testp = Featurewiz_MRMR_Model(model=model, model_type=model_type, encoding_type='ordinal', verbose=1)

In [3]:
testp.fit(X, y)
Xt, yt = testp.transform(X_train,y_train)
print(type(Xt))
print(type(yt))

Model type: Classification

Polars Featurewiz SULOV-MRMR Feature Selection Algorithm
Initial Features: 27
SULOV removed Features (0): 
SULOV selected Features (27): ['surgical_lesion', 'lesion_1', 'temp_of_extremities', 'abdomen', 'pulse', 'pain', 'peristalsis', 'hospital_number', 'peripheral_pulse', 'mucous_membrane', 'nasogastric_reflux', 'abdomo_appearance', 'outcome', 'respiratory_rate', 'abdominal_distention', 'rectal_exam_feces', 'rectal_temp', 'total_protein', 'age', 'capillary_refill_time', 'nasogastric_tube', 'nasogastric_reflux_ph', 'packed_cell_volume', 'abdomo_protein', 'lesion_2', 'lesion_3', 'cp_data']
base importances:  [0.31767067 0.04358303 0.01205622 0.01708942 0.0218404  0.00690538
 0.02017549 0.03742873 0.0219442  0.01332265 0.01877271 0.02362275
 0.09692364 0.02429856 0.04077942 0.01871282 0.01713732 0.05225566
 0.         0.03154579 0.038123   0.03294957 0.01621778 0.04989126
 0.         0.         0.02675354] tier_thresholds[1] 0.05083702132105828
iter limit 5
it

In [4]:
print(len(testp.selected_features))
Xtt = testp.transform(X_test)
type(Xtt)

12


polars.dataframe.frame.DataFrame

In [5]:
y_testt = testp.y_encoder.transform(y_test)
ypt = testp.predict(X_test)
print_classification_metrics(y_testt.to_numpy().ravel(), ypt, verbose=1)

Bal accu 83%
              precision    recall  f1-score   support

           0       0.91      0.72      0.81        29
           1       0.78      0.94      0.85        31

    accuracy                           0.83        60
   macro avg       0.85      0.83      0.83        60
weighted avg       0.85      0.83      0.83        60

final average balanced accuracy score = 0.83


0.8298109010011123

In [7]:
from polars_sulov_mrmr import Sulov_MRMR
#mrmr = Sulov_MRMR(corr_threshold=0.7, verbose=0)
X_pipeline = Pipeline([
    ('datetime_transformer', Polars_DateTimeTransformer(datetime_features=[])), # Specify your datetime columns
    ('cat_transformer', Polars_CategoricalEncoder(encoding_type='ordinal', categorical_features='auto')),
    ('nan_transformer', Polars_MissingTransformer(strategy="median")),
    ('ytransformer', YTransformer()),
    ])
Y_pipeline = Pipeline([
    ('featurewiz', Sulov_MRMR(corr_threshold=0.7, verbose=0)),
    ])
##    Usage missing value fillers
feature_selection = Pipeline([
        ('X_pipeline', X_pipeline),
        ('Y_pipeline', Y_pipeline)
    ])
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted

class testx( TransformerMixin): # Class name 
    def __init__(self, model=model, 
            model_type='classification', encoding_type='target', 
            imputation_strategy='mean', corr_threshold = 0.7,
            verbose = 0):
        self.model = model
        self.model_type = model_type.lower()
        self.encoding_type = encoding_type.lower()
        self.imputation_strategy = imputation_strategy.lower()
        self.corr_threshold = corr_threshold
        self.feature_selection = feature_selection
        self.y_encoder = Polars_ColumnEncoder()

    def fit(self, X, y):
        self.feature_selection.fit(X,y)
        self.y_encoder.fit(y)
        return self

    def transform(self, X, y=None):
        if y is None:
            return self.feature_selection.transform(X)
        else:
            Xt = self.feature_selection.transform(X)
            yt = self.y_encoder.transform(y)
            return Xt, yt

    def fit_transform(self, X, y):
        self.fit(X, y)
        Xt = self.transform(X)
        yt = self.y_encoder.transform(y)
        return Xt, yt

    

In [8]:
model.fit(Xt, yt)

RandomForestClassifier(random_state=99)

In [9]:
base_importances = model.feature_importances_
tier_thresholds = np.percentile(base_importances, [33, 66])
tier_thresholds

array([0.04584887, 0.05855708])

In [10]:
# Stratify features into importance tiers
all_features = sorted(X.columns)
tiers = {
    'high': [f for f, imp in zip(all_features, base_importances) 
            if imp >= tier_thresholds[1]],
    'medium': [f for f, imp in zip(all_features, base_importances) 
            if tier_thresholds[0] <= imp < tier_thresholds[1]],
    'low': [f for f, imp in zip(all_features, base_importances) 
            if imp < tier_thresholds[0]]
}


In [11]:
tiers['high']

['abdomen', 'abdominal_distention', 'lesion_2', 'mucous_membrane']

In [12]:
testp

Featurewiz_MRMR_Model(encoding_type='ordinal',
                      model=RandomForestClassifier(random_state=99), verbose=1)